# Анализ базы данных на основе SQL

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.
Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке.

__Цель работы:__ сформулировать ценностное предложение для нового продукта

__Ход исследования__

Необходимые для исследования данные я получу из таблиц books, authors, publishers, reviews. О качестве данных ничего не известно. Ознаколюсь с таблицами. Опираясь на имеющиеся данные получу ответы на вопросы:  
- сколько книг вышло после 1 января 2000 года;
- для каждой книги посчитаем количество обзоров и среднюю оценку;
- определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- определим автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

## Ознакомление с данными

Выведем 5 сток каждой из таблиц

In [1]:
import pandas as pd
from sqlalchemy import text, create_engine

db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

Выведем таблицу books

In [2]:

query = '''SELECT * FROM books LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Выведем таблицу authors

In [3]:
query = '''SELECT * FROM authors LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Выведем таблицу publishers

In [4]:
query = '''SELECT * FROM publishers LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Выведем таблицу ratings

In [5]:
query = '''SELECT * FROM ratings LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Выведем таблицу reviews

In [6]:
query = '''SELECT * FROM reviews LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


На данном этапе ознакомились с данными и теперь имеем представление о их содержании.

## Задачи

### Посчитаем, сколько книг вышло после 1 января 2000 года

In [7]:
challenge_1 = ''' SELECT COUNT(book_id) AS количество_книг
    FROM books
    WHERE publication_date > '2000-01-01'; ''' 
pd.io.sql.read_sql(sql=text(challenge_1), con = con)

,количество_книг
0,819


In [8]:
challenge_1_1 = ''' SELECT COUNT(DISTINCT title) AS  уникальные_книги
    FROM books
    WHERE publication_date > '2000-01-01'; ''' 
pd.io.sql.read_sql(sql=text(challenge_1_1), con = con)

,уникальные_книги
0,818


Как видим, количество книг, вышедших после 1 января 2000 года составляет 819. Число уникальных книг равно 818, видимо одна книга издавалась 2 раза.

### Для каждой книги посчитаем количество обзоров и среднюю оценку

Для начала проверим уникальные id книг в таблицах.

In [9]:
challenge_2_2 = ''' 

SELECT count(DISTINCT book_id)
       FROM books;'''
pd.io.sql.read_sql(sql=text(challenge_2_2), con = con)

,count
0,1000


In [10]:
challenge_2_3 = ''' 

SELECT count(DISTINCT book_id)
       FROM ratings;'''
pd.io.sql.read_sql(sql=text(challenge_2_3), con = con)

,count
0,1000


In [11]:
challenge_2_3 = ''' 

SELECT count(DISTINCT book_id)
       FROM reviews;'''
pd.io.sql.read_sql(sql=text(challenge_2_3), con = con)

,count
0,994


Как видим, в таблице reviews имеются не все обзоры по имеющимся книгам.

In [12]:
challenge_2_0 = ''' 
WITH 
-- первый подзапрос с псевдонимом i
i AS (SELECT *
      FROM books), 
j AS (SELECT *
      FROM ratings),
u AS (SELECT *
      FROM reviews)
      
SELECT   i.title,
         i.book_id,
         AVG(j.rating) AS средний_рейтинг,
         COUNT(DISTINCT u.review_id) AS количество_обзоров

FROM i JOIN j ON j.book_id = i.book_id LEFT JOIN u ON u.book_id = i.book_id
GROUP BY i.title,
         i.book_id
ORDER BY средний_рейтинг DESC
LIMIT 10;''' 
pd.io.sql.read_sql(sql=text(challenge_2_0), con = con)

,title,book_id,средний_рейтинг,количество_обзоров
0,Arrows of the Queen (Heralds of Valdemar #1),86,5.0,2
1,Crucial Conversations: Tools for Talking When ...,169,5.0,2
2,Alas Babylon,62,5.0,2
3,Act of Treason (Mitch Rapp #9),57,5.0,2
4,Angels Fall,76,5.0,2
5,Captivating: Unveiling the Mystery of a Woman'...,136,5.0,2
6,A Woman of Substance (Emma Harte Saga #1),55,5.0,2
7,A Fistful of Charms (The Hollows #4),20,5.0,2
8,A Dirty Job (Grim Reaper #1),17,5.0,4
9,Dead Souls,182,5.0,2


Максимальный рейтинг равен 5.

In [13]:
challenge_2_0 = ''' 
WITH 
-- первый подзапрос с псевдонимом i
i AS (SELECT *
      FROM books), 
j AS (SELECT *
      FROM ratings),
u AS (SELECT *
      FROM reviews)
      
SELECT   i.title,
         i.book_id,
         AVG(j.rating) AS средний_рейтинг,
         COUNT(DISTINCT u.review_id) AS количество_обзоров

FROM i JOIN j ON j.book_id = i.book_id LEFT JOIN u ON u.book_id = i.book_id
GROUP BY i.title,
         i.book_id
ORDER BY средний_рейтинг
LIMIT 10;''' 
pd.io.sql.read_sql(sql=text(challenge_2_0), con = con)

,title,book_id,средний_рейтинг,количество_обзоров
0,Harvesting the Heart,303,1.500000,2
1,Junky,371,2.000000,2
2,Drowning Ruth,202,2.000000,3
3,His Excellency: George Washington,316,2.000000,2
4,The World Is Flat: A Brief History of the Twen...,915,2.250000,3
5,The Mermaid Chair,794,2.333333,3
6,The Kitchen God's Wife,772,2.333333,3
7,Heretics of Dune (Dune Chronicles #5),313,2.500000,2
8,Amsterdam,72,2.500000,2
9,Dr. Faustus,193,2.500000,2


Минимальный рейтинг равен 1.5

In [14]:
challenge_2_0 = ''' 
WITH 
-- первый подзапрос с псевдонимом i
i AS (SELECT *
      FROM books), 
j AS (SELECT *
      FROM ratings),
u AS (SELECT *
      FROM reviews)
      
SELECT   i.title,
         i.book_id,
         AVG(j.rating) AS средний_рейтинг,
         COUNT(DISTINCT u.review_id) AS количество_обзоров

FROM i JOIN j ON j.book_id = i.book_id LEFT JOIN u ON u.book_id = i.book_id
GROUP BY i.title,
         i.book_id
ORDER BY количество_обзоров DESC
LIMIT 10;''' 
pd.io.sql.read_sql(sql=text(challenge_2_0), con = con)

,title,book_id,средний_рейтинг,количество_обзоров
0,Twilight (Twilight #1),948,3.662500,7
1,The Da Vinci Code (Robert Langdon #2),696,3.830508,6
2,Eat Pray Love,207,3.395833,6
3,The Alchemist,627,3.789474,6
4,The Catcher in the Rye,673,3.825581,6
5,The Curious Incident of the Dog in the Night-Time,695,4.081081,6
6,Harry Potter and the Prisoner of Azkaban (Harr...,302,4.414634,6
7,Harry Potter and the Chamber of Secrets (Harry...,299,4.287500,6
8,Outlander (Outlander #1),497,4.125000,6
9,The Book Thief,656,4.264151,6


Максимальное число обзоров 7.

In [15]:
challenge_2_0 = ''' 
WITH 
-- первый подзапрос с псевдонимом i
i AS (SELECT *
      FROM books), 
j AS (SELECT *
      FROM ratings),
u AS (SELECT *
      FROM reviews)
      
SELECT   i.title,
         i.book_id,
         AVG(j.rating) AS средний_рейтинг,
         COUNT(DISTINCT u.review_id) AS количество_обзоров

FROM i JOIN j ON j.book_id = i.book_id LEFT JOIN u ON u.book_id = i.book_id
GROUP BY i.title,
         i.book_id
ORDER BY количество_обзоров
LIMIT 10;''' 
pd.io.sql.read_sql(sql=text(challenge_2_0), con = con)

,title,book_id,средний_рейтинг,количество_обзоров
0,The Natural Way to Draw,808,3.000000,0
1,Disney's Beauty and the Beast (A Little Golden...,191,4.000000,0
2,Essential Tales and Poems,221,4.000000,0
3,The Cat in the Hat and Other Dr. Seuss Favorites,672,5.000000,0
4,Leonardo's Notebooks,387,4.000000,0
5,Anne Rice's The Vampire Lestat: A Graphic Novel,83,3.666667,0
6,Death: The High Cost of Living,186,3.000000,1
7,Creepshow,165,4.500000,1
8,Debt of Honor (Jack Ryan #7),187,3.000000,1
9,A Christmas Carol and Other Christmas Writings,14,4.500000,1


Как видим, на такие книги как The Natural Way to Draw, Disney's Beauty and the Beast, Essential Tales and Poems, The Cat in the Hat and Other Dr. Seuss Favorites, Leonardo's Notebooks, Anne Rice's The Vampire Lestat: A Graphic Nove не было отзывов, поэтому минимальное количество отзывов 0.

На данном этапе выявили, что было издано 1000 книг, из которых 6 не имеют отзывов. Максимальный рейтинг книг 5, минимальный 1.5. Максимальное число отзывов 7, минимальное 0

### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц.

In [16]:
challenge_3 = ''' 
WITH 

i AS (SELECT publisher_id,
             COUNT(book_id) AS количество_книг
      FROM books
      WHERE num_pages > 50
      GROUP BY publisher_id)
      
SELECT publishers.publisher AS издательство,
       i.количество_книг

FROM i JOIN publishers ON i.publisher_id = publishers.publisher_id 
ORDER BY количество_книг DESC
LIMIT 5;''' 
pd.io.sql.read_sql(sql=text(challenge_3), con = con)

,издательство,количество_книг
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Bantam,19


Как видим, больше всего книг было издано в Penguin Books - 42.

### Определим автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками.

In [17]:
challenge_4 = ''' 
WITH 

i AS (SELECT book_id,
             AVG(rating) AS средняя_оценка
      FROM ratings
      GROUP BY book_id
      HAVING COUNT(book_id) >= 50), 
j AS (SELECT book_id,
             author_id
      FROM books)
      
SELECT authors.author,
       AVG(i.средняя_оценка) AS средняя_оценка
      
FROM i JOIN j ON i.book_id = j.book_id JOIN authors ON j.author_id = authors.author_id
GROUP BY authors.author
ORDER BY средняя_оценка DESC
LIMIT 5;''' 
pd.io.sql.read_sql(sql=text(challenge_4), con = con)

,author,средняя_оценка
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645


Как видим, наибольший средний рейтинг изданных книг имеет J.K. Rowling/Mary GrandPré

### Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [18]:
challenge_5 = ''' 
WITH 

i AS (SELECT username,
             COUNT(username) AS количество_оценок
      FROM ratings
      GROUP BY username
      HAVING COUNT(username) > 48), 
j AS (SELECT username,
             COUNT(username) AS количество_обзоров
      FROM reviews
      GROUP BY username)
      
SELECT  AVG(j.количество_обзоров) AS средняя_оценка
      
FROM i JOIN j ON i.username = j.username;''' 
pd.io.sql.read_sql(sql=text(challenge_5), con = con)

,средняя_оценка
0,24.0


Как видим, среднее количество обзоров равно 24.

На данном этапе сделаны следующие выводы:  
* количество книг, вышедших после 1 января 2000 года составляет 819  
* число уникальных книг, вышедших после 1 января 2000, равно 818, видимо одна книга издавалась 2 раза  
* максимальный рейтинг книг равен 5  
* минимальный рейтинг равен 1.5  
* максимальное число отзывов на книгу 7  
* минимальное количество отзывов на книгу 0  
* больше всего книг было издано в Penguin Books - 42  
* наибольший средний рейтинг изданных книг, которые имеют 50 и более оценок - J.K. Rowling/Mary GrandPré
* среднее количество обзоров пользователей, у которых имеется более 48 оценок книг равно 24
* на такие книги как The Natural Way to Draw, Disney's Beauty and the Beast, Essential Tales and Poems, The Cat in the Hat and Other Dr. Seuss Favorites, Leonardo's Notebooks, Anne Rice's The Vampire Lestat: A Graphic Nove не было отзывов

## Вывод.

__Проделана следующая работа:__  
1. Выполнено ознакомление с данными  
2. Выполнены задачи по расчетам, а именно:  
   * Посчитано, сколько книг вышло после 1 января 2000 года  
   * Для каждой книги посчитано количество обзоров и средняя оценка  
   * Определено издательство, которое выпустило наибольшее число книг толще 50 страниц  
   * Определен автор с самой высокой средней оценкой книг — учитывая только книги с 50 и более оценками  
   * Посчитано среднее количество обзоров от пользователей, которые поставили больше 48 оценок

__Сделаны следующие выводы:__  
* количество книг, вышедших после 1 января 2000 года составляет 819  
* число уникальных книг, вышедших после 1 января 2000, равно 818, видимо одна книга издавалась 2 раза  
* максимальный рейтинг книг равен 5  
* минимальный рейтинг равен 1.5  
* максимальное число отзывов на книгу 7  
* минимальное количество отзывов на книгу 0 
* больше всего книг было издано в Penguin Books - 42  
* наибольший средний рейтинг изданных книг, которые имеют 50 и более оценок - J.K. Rowling/Mary GrandPré
* среднее количество обзоров пользователей, у которых имеется более 48 оценок книг равно 24
* в таблице reviews отсутствуют обзоры на такие книги как The Natural Way to Draw, Disney's Beauty and the Beast, Essential Tales and Poems, The Cat in the Hat and Other Dr. Seuss Favorites, Leonardo's Notebooks, Anne Rice's The Vampire Lestat: A Graphic Nove не было отзывов.